<a href="https://colab.research.google.com/github/kconstable/market_predictions/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning LSTM Models


## Import Libraries 

In [1]:
# install keras tuner
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 5.3 MB/s 


In [2]:
# deep learning /RNN
import tensorflow as tf
from keras import backend as K
from tensorflow import keras
from keras.models import Sequential,backend
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import keras_tuner as kt
from kerastuner.tuners import Hyperband
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# stats, data structures and plotting
import random as rn
import math
import numpy as np
import pandas as pd
import timeit
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
from scipy import stats

# File operations
import os
import pickle
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


# Set Seeds
seed = 1985
np.random.seed(seed)
rn.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Mounted at /content/drive


In [3]:
# check the collab connection
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 26 21:27:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Data
+ Load market data
+ The data was collected, cleaned and consolidated 
+ Optimal features were selected using RFE

In [35]:
stock = 'VMW'
df_vmw = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
df_vmw_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
print(f"\n{stock}")
print("="*60)
print(f"{stock}-all features: ",df_vmw.shape)
print(f"{stock}-optimal features: ",df_vmw_features.shape)


crypto='BTC'
df_btc = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data.pickle')
df_btc_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data_features.pickle')
print(f"\n{crypto}")
print("="*60)
print(f"{crypto}-all features: ",df_btc.shape)
print(f"{crypto}-optimal features: ",df_btc_features.shape)


stock='BLX.TO'
df_blx = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
df_blx_features = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
print(f"\n{stock}")
print("="*60)
print(f"{stock}-all features: ",df_blx.shape)
print(f"{stock}-optimal features: ",df_blx_features.shape)


VMW
VMW-all features:  (2570, 29)
VMW-optimal features:  (2510, 10)

BTC
BTC-all features:  (1000, 28)
BTC-optimal features:  (940, 10)

BLX.TO
BLX.TO-all features:  (2550, 10)
BLX.TO-optimal features:  (2550, 10)


# Data Preprocessing for LSTM Deep Learning


### Prepare the Data 
+ Reverse the date-time index
+ Convert the date-time index to an integer index
+ Convert to numpy arrays

In [5]:
def prepare_data(df,y='close',features=[]):
  """
  Filter, scale and convert dataframe data to numpy arrays

  Inputs: 
    df       => A dataframe of observations with features and y-labels
    y        => The name of the column that is the truth labels
    features => A list of features.  Used to subset columns

  Outputs:
    scaled_y => numpy array of the y-label data
    scaled_x => numpy array of the training features

  """

  # reverse the index such that dates are in chronological order
  df = df.iloc[::-1]

  # Subset features, get the y-label values
  df_y = df[y]
  df_X = df[features]

  # replace the date index with an integer index
  df_X.reset_index(drop=True,inplace=True)

  # convert to numpay arrays
  array_X = np.array(df_X)
  array_y = np.array(df_y).reshape(-1,1)

  # print the output
  print("\nData Preparation")
  print("="*60)
  print(f"=> {len(features)} Features")
  print(f"=> Input Dimensions :{array_X.shape}")
  print(f"=> Output Dimensions:{array_y.shape}")
  print("\n")

  return array_y, array_X


### Split the Data into Train/Test Sets
+ Time series data cannot be split randomly like other observational data because the order is important.  
+ The data is split into train/test based on dates
+ The close price at time t is the target variable, while the close price at t-1 is also included as a feature

In [6]:
def split_train_test(X,idx_close,train_pct=0.80):
  """
  Split the dataset into train/test based on input train percentage
  Input:
    X: time-series dataset of all features in a numpy array
    idx_close:  The index position of the target variable in the dataset
    train_pct: The percent of data allocated to train.  The remainder is test
  Output:
    Two numpay arrays, train_X and test_X
  """

  # Train
  train_num = math.ceil(X.shape[0]*train_pct)
  train_X = X[0:train_num,:]
  train_str = f'{round(train_X.shape[0]/X.shape[0],3)*100}%'

  # Test
  # test_X = X[train_num-n_steps:,:]  # if you use this, need to add n_steps as a param!
  test_X = X[train_num:,:]
  test_str =f'{round(test_X.shape[0]/X.shape[0],3)*100}%'

  # print the results
  print("\nSplit Data (X)")
  print("="*60)
  print(f'Data  :100.0% {X.shape}')
  print(f'Train :{train_str:>6} {train_X.shape}')
  print(f'Test  :{test_str:>6} {test_X.shape}')
  print('\n')

  # create the line plots
  plot_train_test_data(train_X,test_X,idx_close)

  return train_X,test_X


In [7]:
def plot_train_test_data(train_x,test_x,idx_close):
  """
  Plots the close price by date, showing the train/test split
  Input:
    train_x: the training dataset
    test_x:  the test dataset
    idx_close: the index position of the close price
  Output: 
     A time-series plot of close price
  """

  ts0 = list(range(0,train_x.shape[0]))
  ts1 = list(range(train_x.shape[0],train_x.shape[0]+test_x.shape[0]))

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=ts0,y=train_x[:,idx_close],name='Train'))
  fig.add_trace(go.Scatter(x=ts1,y=test_x[:,idx_close],name='Test'))
  fig.add_shape(type ='rect',
                  x0=ts1[0],x1=ts1[-1],
                  y0=0,y1=1,
                  line=dict(color='#F6B28D'),
                  fillcolor='#F6B28D',
                  opacity=0.1)
  fig.add_shape(type ='rect',
                  x0=ts0[0],x1=ts0[-1],
                  y0=0,y1=1,
                  line=dict(color='#7BA1AA'),
                  fillcolor='#7BA1AA',
                  opacity=0.1)
  fig.update_layout(title = 'Train/Test Datasets (Scaled)',
                      template="plotly_white",
                      yaxis_title='Closing Price (Scaled)',
                      xaxis_title='time steps',
                      width = 600,
                      height =600)
  fig.update_shapes(dict(xref='x',yref='paper'))
  fig.show()



### Partition the Data

In [8]:
def create_partitions(data,idx_close,n_steps,n_predict,visualize=False):
  """
  This function partitions the train/test data into batches with times-step windows for training

  Each batch consists of n_steps of training data, and n_predict steps of label data
  The function outputs an x array [samples, time steps, features] and a y array [samples, time steps]

  Inputs:
    data      => train or test array
    idx_close => the position of y value in the data
    n_steps   => the number of time steps in each training batch
    n_predict => the number of time steps that will be predicted
    visualize => boolean, will plot a visual of training/prediction windows
  Outputs:
    array(i)  => np.array of batched & partitioned training data with features
    array(p)  => np.array of batched & partitioned y-lables 

  Reference:
    https://www.relataly.com/time-series-forecasting-multi-step-regression-using-neural-networks-with-multiple-outputs-in-python/5800/
  """
  n = data.shape[0]
  window = n_steps + n_predict
  i, p = [],[]

  # print the moving window
  if visualize:
    print("Data Window: I(Input), P(Predict),-(scanned), +(to be scanned)")
    print("="*max(n,100))

  # create the partitions
  for step in range(n_steps, n-n_predict):
    # get the input window + all features

    # train window
    i.append(data[step-n_steps:step,:])

    # get the prediction window + the closing price
    p.append(data[step:step+n_predict,idx_close])

    # print the moving window
    if visualize and step <= 50:
      scanned = n-((step-n_steps)+window)
      print("-"*(step-n_steps),'I'*n_steps,'P'*n_predict,"+"*scanned,sep="")
    
  return np.array(i),np.array(p)

In [9]:
def plot_training_window(x_array,y_array,idx_close,n_steps,n_predict,batch):
  """
  Plots a single training batch showing the train/prediction windows
  Inputs: 
    x_array/y_array => partitioned data from create_partitions
    idx_close       => the position of the y-label (close price)
    n_steps         => the number of time steps in each training batch
    n_predict       => the number of time steps predicted each training batch
    batch           => the batch number to plot
  Outputs:
    A line chart with the train/predict values
  """
  # convert the arrays to dataframes
  # align the x indexes to compare
  df_y = pd.DataFrame(y_array[batch],index=range(n_steps-1,n_predict+n_steps-1),columns=['y'])
  df_x = pd.DataFrame(x_array[batch+1])[idx_close]
  df_x = pd.DataFrame(df_x)
  df_x.columns = ['x']

  # create the plots
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df_x.index,y=df_x['x'],name='train window'))
  fig.add_trace(go.Scatter(x=df_y.index,y=df_y['y'],name='predict window'))
  fig.update_layout(template='plotly_white',
                    title='Train/Predict Windows',
                    yaxis_title = 'Closing Price (Scaled)',
                    xaxis_title='Period',
                    width = 600,
                    height = 600)
  fig.show()


### Data Transformations

#### Check for stationarity
+ Deep learning performs better with stationary data where the mean and variance is constant over time.  Market price data is rarely stationary as it includes trending. The time-series data is transformed to make it stationary 
+ The transform_stationary funciton applies data transformations and performs the Augmented Dickey-Fuller (ADF) to test for stationarity
   null hypothesis: the data is non-stationary
  + alternative: the data is stationary
  + => if the test stat is less than all critical values,  we cannot reject the null, therefore the data is non-stationary

**Reference**
+ [Checking time-series data for stationarity](https://analyzingalpha.com/check-time-series-stationarity-python#augmented-dickey-fuller-adf)

In [10]:
def transform_stationary(df,features_to_transform,transform='log'):
  """
  Transform time-series data using a log or boxcox transform.  Calculate the augmented
  dickey-fuller (ADF) test for stationarity after the transform
  Inputs:
    df: a dataframe of features
    features_to_transform: A list of features to apply the transform
    transform: The transform to apply (log, boxbox)
  Output
    Applies the transforms inplace in df
  """
  # transform each column in the features_to_transform list
  for feature in df.columns:
    if feature in features_to_transform:
      # log transform
      if transform=='log':
        df[feature] = df[feature].apply(np.log)

      # boxcox transform  
      elif transform=='boxcox':
        bc,_ = stats.boxcox(df[feature])
        df[feature] = bc

      else:
        print("Transformation not recognized")

  # check the closing price for stationarity using the augmented dicky fuller test
  t_stat, p_value, _, _, critical_values, _  = adfuller(df.close.values, autolag='AIC')
  print('Augmented Dicky Fuller Test for Stationarity')
  print("="*60)
  print(f'ADF Statistic: {t_stat:.2f}')
  for key, value in critical_values.items():
    print('Critial Values:')
    if t_stat < value:
      print(f'   {key}, {value:.2f} => non-stationary')
    else:
      print(f'   {key}, {value:.2f} => stationary')

### Process the Data
+ Select the dataset (BTC,VMW,BLX.TO) for all features or the features selected using RFE
+ Transform the data to be stationary
+ Convert to numpy arrays
+ Scale the data for deep learning
+ Split the data into train/test sets
+ Partition the data into time-series windows
+ Plot the close price split into train/test sets
+ Plot a single time-series train/predict batch

In [36]:
# Training Config

# uncomment the desired data
# df = df_btc.copy()
# df = df_btc_features.copy()
# df = df_vmw.copy()
df = df_vmw_features.copy()
# df = df_blx.copy()
# df = df_blx_features.copy()


# Extract the features needed for training
features = [c for c in df.columns if c not in ['symbol']]
n_steps  = 40   # The number of time steps included in each training batch
n_predict = 5   # The number of time steps into the future the model will predict
idx_close = df.columns.get_loc("close") # index postiion variable (closing price)
transform = 'log'

# Transform the data to be stationary
features_to_transform = ['open','high','low','close'] 
transform_stationary(df,features_to_transform,transform)


# Convert to numpy arrays
array_y,array_X = prepare_data(df,'close',features)


# scale the input and outputs
scaler_X = MinMaxScaler(feature_range=(0,1))
scaler_y = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler_X.fit_transform(array_X)
scaled_y = scaler_y.fit_transform(array_y)


# split into train, test
train_pct = 0.80
train_x,test_x=split_train_test(scaled_X,idx_close,train_pct)


# Partition the train/test data into time series windows for training
# LSTM input format: [samples, time steps, features]
x_train, y_train = create_partitions(train_x,idx_close,n_steps, n_predict)
x_test,  y_test  = create_partitions(test_x, idx_close,n_steps, n_predict)


# Print the results
print('Train/Test Dimensions')
print('='*60)
print("Train Data Dimensions: ","x",x_train.shape,"y",y_train.shape)
print("Test Data Dimensions : ","x",x_test.shape," y",y_test.shape)

print("\n\nCheck the dimensions of the training windows")
print("="*60)
print('The last value in the training batch should match the first value in the predict batch')
print("Last Training Value:   ",x_train[1][n_steps-1][idx_close])
print("First Prediction Value:",y_train[0][0])

# Make sure the train/predict batch windows are aligned
plot_training_window(x_train,y_train,idx_close,n_steps,n_predict,seed//10)



Augmented Dicky Fuller Test for Stationarity
ADF Statistic: -1.62
Critial Values:
   1%, -3.43 => stationary
Critial Values:
   5%, -2.86 => stationary
Critial Values:
   10%, -2.57 => stationary

Data Preparation
=> 10 Features
=> Input Dimensions :(2510, 10)
=> Output Dimensions:(2510, 1)



Split Data (X)
Data  :100.0% (2510, 10)
Train : 80.0% (2008, 10)
Test  : 20.0% (502, 10)




Train/Test Dimensions
Train Data Dimensions:  x (1963, 40, 10) y (1963, 5)
Test Data Dimensions :  x (457, 40, 10)  y (457, 5)


Check the dimensions of the training windows
The last value in the training batch should match the first value in the predict batch
Last Training Value:    0.46099671803114717
First Prediction Value: 0.46099671803114717


# Deep Learning LSTM

### Build the LSTM Model

In [12]:
def create_lstm_model(config,x_train,lr=0.001):
  """
  Builds an LSTM deep learning model
  Inputs:
    config: A dictionary containing the network topology
    x_train: The train set with all features
    lr: the learning rate for the adam optimizer
  Output:
    An LSTM model built according to config
  """
  # get window size
  n_steps = config['data']['n_steps']
  n_predict = config['data']['n_predict']
  n_features = x_train.shape[2]

  # clear previous models
  backend.clear_session()

  # LSTM Model+ first layer
  model = Sequential(name='LSTM')
  model.add(LSTM(n_steps,return_sequences=True,input_shape=(n_steps,n_features)))

  # add additional layers
  for layer,nodes,ret_seq,drop in config['layers']:
    # add LSTM layers + dropout
    if layer=='lstm':
      model.add(LSTM(nodes,return_sequences =ret_seq))
      if drop is not None:
        model.add(Dropout(drop))

    # Add Dense Layers + dropout
    elif layer =='dense':
      model.add(Dense(nodes))
      if drop is not None:
        model.add(Dropout(drop))

  # add the prediction layer
  model.add(Dense(n_predict))
  
  # compile
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mse')
  model.summary()

  return model


### Calculate Model Performance

In [13]:
def calculate_performance(x_test,y_test,model,scaler_y):
  """
  Predicts the price output and calculates the performance of the model
  Inputs:
    x_test: the test dataset
    y_test: the test labels
    model: the trained LSTM model
    scaler_y: the transfrom applied to the labels
  Output:
    Prints the performance metrics and returns the predictions
  """

  # Predict the prices
  y_pred = model.predict(x_test)

  # convert units back to the original scale
  y_pred_unscaled = scaler_y.inverse_transform(y_pred)
  y_test_unscaled = scaler_y.inverse_transform(y_test)

  # # Root mean squarred error,mean abs error, mean abs percent error, median abs percent error
  rmse  = math.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
  mae   = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
  mape  = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred_unscaled)/ y_test_unscaled))) * 100
  mdape = np.median((np.abs(np.subtract(y_test_unscaled, y_pred_unscaled)/ y_test_unscaled)) ) * 100

  print("\nModel Error")
  print("="*62)
  print(f'{"Mean Absolute Error (MAE)" :-<55} {np.round(mae, 2):>5}')
  print(f'{"Root Mean Squared Error (MSE)" :-<55} {np.round(rmse,2):>5}')
  print(f'{"Mean Absolute Percentage Error (MAPE)" :-<55} {np.round(mape, 2):>5}%')
  print(f'{"Median Absolute Percentage Error (MDAPE)" :-<55} {np.round(mdape, 2):>5}%')

  return y_pred


### Plot Training Metrics

In [14]:

def plot_training_metrics(history):
  """
  Plots the training metrics: loss and val_loss by epochs
  Input:
    history: the train history
  """
  # get the number of epochs
  epochs = list(range(1, len(history.history['loss']) + 1))

  # min_train_loss = min(history.history['loss'])
  # min_val_loss = min(history.history['val_loss'])

  # create the line plots
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=epochs,
                           y=history.history['loss'],
                           name = 'train-loss',
                           line=dict(width=3,color='royalblue')))
  fig.add_trace(go.Scatter(x=epochs,
                           y=history.history['val_loss'],
                           name='val-loss',
                           line=dict(width=3,color='crimson')))

  fig.update_layout(title = 'Training Metrics',
                    template="plotly_white",
                    width = 700,
                    height= 500,
                    yaxis_title='loss',
                    xaxis_title='epochs')

  fig.show()

## Plot Predicted vs Actual Price

In [15]:
def plot_price_predictions(batch, idx_close, x_test, y_pred_scaled,scaler_y): 
  """
  Plots the predicted vs actual prices for a single batch
  Inputs:
    batch: the batch index to plot
    idx_close: the index position of the close price in the feature set
    x_test: the test dataset
    y_pred_scaled: the predicted y values (scaled)
    scaler_y: the scaler used to scale y
  """

  # unscale the y predictions
  y_pred_unscaled = scaler_y.inverse_transform(y_pred_scaled)

  # unscale the x_test data
  x_test_np = np.array(pd.DataFrame(x_test[batch])[idx_close]).reshape(-1,1)
  x_test_unscaled = scaler_y.inverse_transform(x_test_np)
  x_test_df = pd.DataFrame(x_test_unscaled)

  # set the indexes for plotting
  max_test_idx=x_test_df.shape[0]
  max_pred_idx =y_pred_unscaled[0].shape[0]
  test_idx = list(range(batch,batch + max_test_idx))
  pred_idx = list(range(batch + max_test_idx,batch + max_test_idx + max_pred_idx))


  # combine the actual + predicted prices
  data = pd.DataFrame(list(zip(y_pred_unscaled[batch], x_test_df[0])), columns=['pred', 'actual'])

  # create the plot
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_idx, y=x_test_df[0],
                        mode='lines',
                        name='Test Data',
                        fill='tozeroy',
                        line_color='#ccc'))
  fig.add_trace(go.Scatter(x=pred_idx, y=data['actual'],
                        mode='lines+markers', 
                        name='Actual Price',
                        fill='tozeroy',
                        line_color ='#ccc')) 
  fig.add_trace(go.Scatter(x=pred_idx, y=data['pred'],
                        mode='lines+markers',
                        name='Predicted Price',
                        line_color='red'))

  fig.update_layout(template = 'plotly_white',
                      title= 'Actual vs Predicted Price',
                      xaxis_title = 'Batch',
                      yaxis_title = 'Price',
                      width=600,
                      height=400)

  fig.show()
  

In [16]:
def plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,n_batches=2):
  """
  Plots random batches of predictions
  """

  # select random batch numbers
  batches=[]
  [batches.append(rn.randint(0,y_pred_scaled.shape[0]-1)) for r in list(range(n_batches))]

  # plot predictions
  for batch in batches:
    plot_price_predictions(batch, idx_close, x_test, y_pred_scaled,scaler_y)


## Train the Model
Training config file.

In [24]:
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',256,True,0.3),('lstm',128,False,0.2),('dense',32,None,0.1),('dense',16,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}
config

{'data': {'n_predict': 5, 'n_steps': 40},
 'layers': [('lstm', 256, True, 0.3),
  ('lstm', 128, False, 0.2),
  ('dense', 32, None, 0.1),
  ('dense', 16, None, 0.1)],
 'train': {'batch_size': 128,
  'early_stop': <keras.callbacks.EarlyStopping at 0x7f7b90c30d50>,
  'epochs': 250}}

### Bitcoin
+ train with all features
+ train with the optimal features

In [ ]:
# BTC - all features
modelname = f'model_{crypto}_{transform}'
model = create_lstm_model(config,x_train)

# Train the model
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')


# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all/train_history.csv')


Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            10880     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           304128    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BTC_log_all/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BTC_log_all/assets


In [ ]:
# BTC -optimal features
modelname = f'model_{crypto}_{transform}'
model = create_lstm_model(config,x_train)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8800      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           304128    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BTC_log_features/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BTC_log_features/assets


### VM Ware
+ train with all features
+ train with optimal features

In [ ]:
# VW Ware -all features
stock= 'VWM'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )


# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            11040     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           304128    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_VWM_log_all/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_VWM_log_all/assets


In [ ]:
# VW Ware -optimal features
stock= 'VWM'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)

# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           304128    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_VWM_log_features/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_VWM_log_features/assets


### BLX.TO

In [ ]:
# BLX  -all features
stock= 'BLX.TO'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            11040     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 256)           304128    
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_all/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_all/assets


In [ ]:
# BLX  -optimal features
stock= 'BLX.TO'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_features/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_features/assets


## Load Saved Model

In [4]:
# get saved training evaluation
modelname = 'model_BTC_log_features'
model = keras.models.load_model(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}')
df_hist = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}/train_history.csv')

# Hyperparameter Tuning

In [ ]:
def build_tuner_model(hp):
  """
  Builds an LSTM model for hyper-parameter tuning using keras-tuner
  """
  # Clear previous session
  backend.clear_session()

  # searchable parameters
  lstm_nodes    = hp.Choice('lstm_units',values=LSTM_UNITS)
  dense_nodes   = hp.Choice('dense_units',values=DENSE)
  dropout_lstm  = hp.Choice('lstm_dropout',values=DROPOUT_LSTM)
  dropout_dense = hp.Choice('dense_dropout',values=DROPOUT_DENSE)
  learn_rate    = hp.Choice('learn_rate',values = LEARN_RATE)


  # LSTM Model+ first layer
  model = Sequential()
  model.add(LSTM(N_STEPS,return_sequences=True,input_shape=(N_STEPS,N_FEATURES)))


  # add layers
  model.add(LSTM(units=lstm_nodes,return_sequences=True))
  model.add(Dropout(dropout_lstm))
  model.add(LSTM(lstm_nodes//2,return_sequences=False))
  model.add(Dropout(max(0.1,dropout_lstm//2)))
  model.add(Dense(units=dense_nodes))
  model.add(Dropout(dropout_dense))
  model.add(Dense(dense_nodes//2))
  model.add(Dropout(max(0.1,dropout_dense//2)))

  # add prediction layer
  model.add(Dense(5))

  # compile
  model.compile(loss='mse',optimizer =tf.keras.optimizers.Adam(learning_rate=learn_rate))
  model.summary()

  return model

#### BTC - Tuning

In [ ]:
# batch_size
# BTC - tuned with optimzed features

# Searchable hyperparameters
LSTM_UNITS    = [256,512,1024,2048]
DENSE         = [128,256,512,1024,2048]
DROPOUT_LSTM  = [0.1,0.20,0.3,0.50]
DROPOUT_DENSE = [0.1,0.20,0.3,0.50]
N_FEATURES    = x_train.shape[2]
N_STEPS       = config['data']['n_steps']
N_PREDICT     = config['data']['n_predict']
LEARN_RATE    = [0.001,0.0001]

# initialize
hp = kt.HyperParameters()


# Setup the Tuner
tuner = Hyperband(
    build_tuner_model,
    max_epochs = 50,
    objective = 'val_loss', 
    directory = 'tune_hyperband',
    project_name = 'prices_tune_hyperband',
    overwrite = True
)

# Implement early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


# Perform hyper
tuner.search(x_train,
             y_train,
             batch_size = 128,
             epochs = 50,
             validation_split=0.10,
             callbacks=[stop_early]
)



# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("Hyperparmeter Search")
print("="*60)
print(f"First LSTM Node Size:          {best_hps.get('lstm_units')}")
print(f"First LSTM Dropout Rate:       {best_hps.get('lstm_dropout')}")
print(f"Second LSTM Node Size:         {best_hps.get('lstm_units')//2}")
print(f"Second LSTM Dropout Rate:      {max(0.1,best_hps.get('lstm_dropout')//2)}")
print(f"First Dense Node Size:         {best_hps.get('dense_units')}")
print(f"First Dense Node Droput Rate:  {best_hps.get('dense_dropout')}")
print(f"Second Dense Node Size:        {best_hps.get('dense_units')//2}")
print(f"Second Dense Node Dropout Rate:{max(0.1,best_hps.get('dense_dropout')//2)}")
print(f"Learning Rate:                 {best_hps.get('learn_rate')}")


print('\n\n')
# show the optimal hyperparameters
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values


Trial 90 Complete [00h 01m 39s]
val_loss: 0.0005997736006975174

Best val_loss So Far: 0.00035921321250498295
Total elapsed time: 00h 32m 06s
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8800      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (D

{'dense_dropout': 0.2,
 'dense_units': 256,
 'learn_rate': 0.001,
 'lstm_dropout': 0.3,
 'lstm_units': 1024,
 'tuner/bracket': 0,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 0,
 'tuner/round': 0}

#### VM Ware Tuning

In [ ]:
# batch_size
# VM Ware - tuned with all features

# Searchable hyperparameters
LSTM_UNITS    = [256,512,1024,2048]
DENSE         = [128,256,512,1024,2048]
DROPOUT_LSTM  = [0.1,0.20,0.3,0.50]
DROPOUT_DENSE = [0.1,0.20,0.3,0.50]
N_FEATURES    = x_train.shape[2]
N_STEPS       = config['data']['n_steps']
N_PREDICT     = config['data']['n_predict']
LEARN_RATE    = [0.001,0.0001]

# initialize
hp = kt.HyperParameters()


# Setup the Tuner
tuner = Hyperband(
    build_tuner_model,
    max_epochs = 50,
    objective = 'val_loss', 
    directory = 'tune_hyperband',
    project_name = 'prices_tune_hyperband',
    overwrite = True
) 

# Implement early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


# Perform hyper
tuner.search(x_train,
             y_train,
             batch_size = 128,
             epochs = 50,
             validation_split=0.10,
             callbacks=[stop_early]
)



# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("Hyperparmeter Search")
print("="*60)
print(f"First LSTM Node Size:          {best_hps.get('lstm_units')}")
print(f"First LSTM Dropout Rate:       {best_hps.get('lstm_dropout')}")
print(f"Second LSTM Node Size:         {best_hps.get('lstm_units')//2}")
print(f"Second LSTM Dropout Rate:      {max(0.1,best_hps.get('lstm_dropout')//2)}")
print(f"First Dense Node Size:         {best_hps.get('dense_units')}")
print(f"First Dense Node Droput Rate:  {best_hps.get('dense_dropout')}")
print(f"Second Dense Node Size:        {best_hps.get('dense_units')//2}")
print(f"Second Dense Node Dropout Rate:{max(0.1,best_hps.get('dense_dropout')//2)}")
print(f"Learning Rate:                 {best_hps.get('learn_rate')}")


print('\n\n')
# show the optimal hyperparameters
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

Trial 90 Complete [00h 00m 40s]
val_loss: 0.0020608254708349705

Best val_loss So Far: 0.0014229147927835584
Total elapsed time: 00h 30m 08s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            11040     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

{'dense_dropout': 0.5,
 'dense_units': 256,
 'learn_rate': 0.001,
 'lstm_dropout': 0.2,
 'lstm_units': 1024,
 'tuner/bracket': 1,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/round': 1,
 'tuner/trial_id': '5a0e9659c9458e8681d2a9a16f0986dc'}

#### BLX Tuning

In [ ]:

# BLX.TO -tuned with optimal features

# Searchable hyperparameters
LSTM_UNITS    = [256,512,1024,2048]
DENSE         = [128,256,512,1024,2048]
DROPOUT_LSTM  = [0.1,0.20,0.3,0.50]
DROPOUT_DENSE = [0.1,0.20,0.3,0.50]
N_FEATURES    = x_train.shape[2]
N_STEPS       = config['data']['n_steps']
N_PREDICT     = config['data']['n_predict']
LEARN_RATE    = [0.001,0.0001]

# initialize
hp = kt.HyperParameters()

# Setup the Tuner
tuner = Hyperband(
    build_tuner_model,
    max_epochs = 50,
    objective = 'val_loss', 
    directory = 'tune_hyperband',
    project_name = 'prices_tune_hyperband',
    overwrite = True
)

# Implement early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


# Perform hyper
tuner.search(x_train,
             y_train,
             batch_size = 128,
             epochs = 50,
             validation_split=0.10,
             callbacks=[stop_early]
)



# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("Hyperparmeter Search")
print("="*60)
print(f"First LSTM Node Size:          {best_hps.get('lstm_units')}")
print(f"First LSTM Dropout Rate:       {best_hps.get('lstm_dropout')}")
print(f"Second LSTM Node Size:         {best_hps.get('lstm_units')//2}")
print(f"Second LSTM Dropout Rate:      {max(0.1,best_hps.get('lstm_dropout')//2)}")
print(f"First Dense Node Size:         {best_hps.get('dense_units')}")
print(f"First Dense Node Droput Rate:  {best_hps.get('dense_dropout')}")
print(f"Second Dense Node Size:        {best_hps.get('dense_units')//2}")
print(f"Second Dense Node Dropout Rate:{max(0.1,best_hps.get('dense_dropout')//2)}")
print(f"Learning Rate:                 {best_hps.get('learn_rate')}")


print('\n\n')
# show the optimal hyperparameters
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

Trial 90 Complete [00h 01m 11s]
val_loss: 0.0002812985621858388

Best val_loss So Far: 0.00015881244326010346
Total elapsed time: 00h 31m 34s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

Hyperparmeter Search
First LSTM Node Size:          1024
First LSTM Dropout Rate:       0.3
Second LSTM Node Size:         512
Second LSTM Dropout Rate:      0.1
First Dense Node Size:         512
First Dense Node Droput Rate:  0.3
Second Dense Node Size:        256
Second Dense Node Dropout Rate:0.1
Learning Rate:                 0.001





{'dense_dropout': 0.3,
 'dense_units': 512,
 'learn_rate': 0.001,
 'lstm_dropout': 0.3,
 'lstm_units': 1024,
 'tuner/bracket': 1,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/round': 1,
 'tuner/trial_id': 'f2129a3a6e372abbf79011d380f1ef82'}

### Search for batch size
+ **BTC**: 64 optimal batch size
+ **VMW**:  32/64 optimal batch size
+ **BLX.TO**: 64/128 optimal batch size

In [ ]:
# optimized hyperparams for BTC with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.3),('lstm',512,False,0.1),('dense',256,None,0.2),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}

# test batch sizes
batch_size = [32,64,128,256,512,1024]

# BTC -optimal features
model = create_lstm_model(config,x_train,n_predict,lr=0.001)

# test batch sizes for performance
for batch in batch_size:

  # train
  history = model.fit(x_train,
                      y_train,
                      batch_size=batch,
                      epochs=config['train']['epochs'],
                      callbacks = [config['train']['early_stop']],
                      validation_split =0.10, 
                      verbose = 0
                      )

  print(f"\n\nBatch Size: {batch}")
  # predict prices
  # calculate the performance metrics
  y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)



Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

In [ ]:
# optimized hyperparams for BLX.TO with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.3),('lstm',512,False,0.1),('dense',256,None,0.2),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}

# test batch sizes
batch_size = [32,64,128,256,512,1024]

# BTC -optimal features
model = create_lstm_model(config,x_train,n_predict,lr=0.001)

# test batch sizes for performance
for batch in batch_size:

  # train
  history = model.fit(x_train,
                      y_train,
                      batch_size=batch,
                      epochs=config['train']['epochs'],
                      callbacks = [config['train']['early_stop']],
                      validation_split =0.10, 
                      verbose = 0
                      )

  print(f"\n\nBatch Size: {batch}")
  # predict prices
  # calculate the performance metrics
  y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)



Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

In [ ]:
# optimized hyperparams for VMW with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.2),('lstm',512,False,0.1),('dense',256,None,0.5),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}

# test batch sizes
batch_size = [32,64,128,256,512,1024]

# BTC -optimal features
model = create_lstm_model(config,x_train,n_predict,lr=0.001)

# test batch sizes for performance
for batch in batch_size:

  # train
  history = model.fit(x_train,
                      y_train,
                      batch_size=batch,
                      epochs=config['train']['epochs'],
                      callbacks = [config['train']['early_stop']],
                      validation_split =0.10, 
                      verbose = 0
                      )

  print(f"\n\nBatch Size: {batch}")
  # predict prices
  # calculate the performance metrics
  y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)


Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            11040     
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

## Re-Train with Optimized Hyperparmeters

### Bitcoin

In [ ]:
# optimized hyperparams for BTC with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.3),('lstm',512,False,0.1),('dense',256,None,0.2),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':64,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}

# BTC -optimal features
crypto ='BTC'
modelname = f'model_{crypto}_{transform}'
model = create_lstm_model(config,x_train,n_predict,lr=0.001)


# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)

# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features_optimized')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features_optimized/train_history.csv')


Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

### VM Ware

In [ ]:
# VW Ware -all features
backend.clear_session()
# optimized hyperparams for VMW with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.2),('lstm',512,False,0.1),('dense',256,None,0.5),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}


stock= 'VWM'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train,n_predict)

# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )


# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all_optimized')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_all_optimized/train_history.csv')

### Boralex

In [ ]:
# BLX.TO -optimal features

# optimized hyperparams for BLX.TO with feature selection
config ={
    'data': {'n_steps':n_steps,'n_predict':n_predict},
    'layers': [('lstm',1024,True,0.3),('lstm',512,False,0.1),('dense',256,None,0.2),('dense',128,None,0.1)],
    'train':{'epochs':250,'batch_size':128,'early_stop':EarlyStopping(monitor='loss',patience=10)}
}


stock = 'BLX.TO'
modelname = f'model_{stock}_{transform}'
model = create_lstm_model(config,x_train,n_predict,lr=0.001)


# train
tic = timeit.default_timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=config['train']['batch_size'],
                    epochs=config['train']['epochs'],
                    callbacks = [config['train']['early_stop']],
                    validation_split =0.10, 
                    verbose = 0
                    )

# print the training time
toc =timeit.default_timer()
print('\nTraining Time')
print('='*60)
print(f'Minutes:{round((toc-tic)/60,2)}\n')

# predict prices
# calculate the performance metrics
# plot training metrics
# plot predictions for random batches
y_pred_scaled = calculate_performance(x_test,y_test,model,scaler_y)
plot_training_metrics(history)
plot_n_price_predictions(idx_close,x_test,y_pred_scaled,scaler_y,2)


# save model and training performance
model.save(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features_optimized')

df_hist = pd.DataFrame(history.history) 
df_hist.to_csv(f'/content/drive/MyDrive/Colab Notebooks/models/{modelname}_features_optimized/train_history.csv')

Model: "LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 40)            8160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 1024)          4362240   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0      

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_features_optimized/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/models/model_BLX.TO_log_features_optimized/assets
